In [ ]:
import pandas as pd
import google.generativeai as genai
import time
import os
from tqdm import tqdm
import json
# Cấu hình API Key
genai.configure(api_key="GEMINI_API")  # <-- Nhớ điền API key ở đây

# Khởi tạo model
model = genai.GenerativeModel(model_name='models/gemini-1.5-flash-latest')

# File input và output
input_file = "output_chunks.xlsx"
output_file = "gen_dental_data_output.xlsx"

# Đọc dữ liệu đầu vào
try:
    df_input = pd.read_excel(input_file)
    df_input = df_input[['Text']].dropna().reset_index(drop=True)
    print(f"Đã đọc file Excel với {len(df_input)} dòng văn bản.")
except Exception as e:
    print(f"Lỗi khi đọc file Excel: {str(e)}")
    exit()

# Tạo DataFrame kết quả
result_columns = [
    "icd_10", "question", "symptom", "diagnosis",
    "document/title", "document/tool", "document/description",
    "cme/title", "cme/tool", "cme/description"
]
if os.path.exists(output_file):
    df_result = pd.read_excel(output_file)
else:
    df_result = pd.DataFrame(columns=result_columns)

# Prompt Template mới
prompt_template = """
Bạn là một chuyên gia y tế chuyên về nha khoa. Dựa vào đoạn văn sau:

"{dental_text}"

Hãy tạo ra một đoạn dữ liệu JSON gồm TẤT CẢ các trường sau. Không được bỏ trống bất kỳ trường nào. Nếu thông tin không rõ ràng, hãy **giả định hợp lý** hoặc điền là "Chưa xác định":

- "icd_10": Mã bệnh tương ứng (ví dụ: "K02", "K05")
- "question": Câu hỏi liên quan đến đoạn văn, tập trung vào nha khoa
- "symptom": Các triệu chứng nha khoa mô tả hoặc giả định
- "diagnosis": Chẩn đoán y khoa liên quan đến nha khoa
- "document/title": Tên tài liệu y khoa liên quan
- "document/tool": Công cụ hỗ trợ chẩn đoán hoặc điều trị
- "document/description": Mô tả tài liệu
- "cme/title": Tên khóa học CME liên quan
- "cme/tool": Công cụ trong khóa học
- "cme/description": Mô tả khóa học

Chỉ trả lời bằng JSON, KHÔNG thêm mô tả hoặc đoạn văn nào khác.
"""


# Hàm sinh dữ liệu
def generate_with_retry(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            if response.text:
                return response
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5
                print(f"Lỗi, thử lại sau {wait_time}s... (Lần {attempt + 1})")
                time.sleep(wait_time)
            else:
                print(f"Lỗi sau {max_retries} lần thử: {str(e)}")
                return None
    return None

# Hàm xử lý và lưu
def process_and_save(response):
    try:
        cleaned = response.text.replace('```json', '').replace('```', '').strip()
        data = json.loads(cleaned)
        global df_result
        new_row = {
            "icd_10": data.get("icd_10", ""),
            "question": data.get("question", ""),
            "symptom": data.get("symptom", ""),
            "diagnosis": data.get("diagnosis", ""),
            "document/title": data.get("document/title", ""),
            "document/tool": data.get("document/tool", ""),
            "document/description": data.get("document/description", ""),
            "cme/title": data.get("cme/title", ""),
            "cme/tool": data.get("cme/tool", ""),
            "cme/description": data.get("cme/description", "")
        }
        df_result.loc[len(df_result)] = new_row
        return True
    except Exception as e:
        print(f"\nLỗi xử lý response: {str(e)}")
        print(f"Response gốc: {response.text[:200]}...")
        return False

# Thiết lập số dòng xử lý
start_row = len(df_result)
end_row = len(df_input)
processed_count = 0
start_time = time.time()

# Vòng lặp xử lý
for index in tqdm(range(start_row, end_row)):
    row = df_input.iloc[index]
    retries = 0

    while retries < 3:
        try:
            dental_text = row['Text'].strip()
            prompt = prompt_template.format(dental_text=dental_text)
            response = generate_with_retry(prompt)

            if response and process_and_save(response):
                processed_count += 1
                df_result.to_excel(output_file, index=False)
                time.sleep(2)
                break
            else:
                retries += 1
                print(f"Retry dòng {index} - Lần {retries}")
                time.sleep(3)

        except Exception as e:
            print(f"\nLỗi dòng {index}: {str(e)}")
            retries += 1
            time.sleep(3)

# Lưu cuối
df_result.to_excel(output_file, index=False)
total_time = (time.time() - start_time) / 60
print(f"\nHoàn thành! Đã xử lý {processed_count} dòng.")
print(f"Thời gian chạy: {total_time:.2f} phút")
print(f"File kết quả: {output_file}")


Đã đọc file Excel với 27 dòng văn bản.


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:05<02:09,  5.64s/it]


KeyboardInterrupt: 